# Dědičnost

Dědičnost v objektově orientovaném programování umožňuje novým třídám převzít vlastnosti a chování od již existujících tříd. Hlavním účelem dědičnosti je zvýšení znovupoužitelnosti kódu a vytváření vztahů mezi třídami. Díky dědičnosti můžeme vytvářet sofistikovanější a organizovanější systémy bez zbytečné duplikace kódu, což vede k lepší správě a udržitelnosti softwarových projektů.

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"Name: {self.name}, Age: {self.age}"

class Employee(Person):
    def __init__(self, name, age, job_title, salary):
        super().__init__(name, age)
        self.job_title = job_title
        self.salary = salary
        
    def payout(self):
        print(f"paying {self.salary} to {self.name}")


class GoodEmployee(Employee):
    def payout(self):
        print(f"paying {self.salary + 10000} to {self.name}")        
        
person = Person("Václav Alt", 31)
employee = Employee("Jindřich Sádlo", 28, "instalatér", 45000)
good_employee = GoodEmployee("Viktor Hroutil", 29, "lepší instalatér", 45000)

# person.payout()
employee.payout()
print(person)
print(employee)
print(good_employee)

In [ ]:
def payout_employess(employees: list[Employee]):
    for employee in employees:
        employee.payout()
        
def send_annoying_mail(persons: list[Person]):
    for person in persons:
        print(f"sending annoying mail to {person.name}")
        
send_annoying_mail([person, employee])

try:
    payout_employess([person, employee])
except AttributeError as e:
    print(e)
    print("You can not payout an ordinary person")

In [ ]:
def payout_employess(employees: list[Employee]):
    for employee in employees:
        if type(employee) == Employee:
            employee.payout()
        
def send_annoying_mail(persons: list[Person]):
    for person in persons:
        print(f"sending annoying mail to {person.name}")
        
send_annoying_mail([person, employee])
payout_employess([person, employee, good_employee])

In [ ]:
def payout_employess(employees: list[Employee]):
    for employee in employees:
        if isinstance(employee, Employee):
            employee.payout()
        
def send_annoying_mail(persons: list[Person]):
    for person in persons:
        print(f"sending annoying mail to {person.name}")
        
send_annoying_mail([person, employee])
payout_employess([person, employee, good_employee])

## Method overriding (překrývání metod)

Překrýváním se označuje to, když potomek třídy definuje vlastní implementaci některé metody předka. Může buď původní implementaci volat (např. přes konstrukci `super()`) a vykonat něco navíc, nebo může předchozí implementaci zcela nahradit. Např. v C++ je nutné metody označit klíčovým slovem `virtual`, aby bylo možné je překrýt, v Pythonu takový mechanismus není.

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return f"Name: {self.name}, Age: {self.age}"

class Employee(Person):
    def __init__(self, name, age, job_title, salary):
        super().__init__(name, age)
        self.job_title = job_title
        self.salary = salary
        
    def payout(self):
        print(f"paying {self.salary} to {self.name}")

    def __str__(self):
        person_info = super().__str__()
        return f"{person_info}, Job Title: {self.job_title}, Salary: {self.salary}"

person = Person("Václav Alt", 31)
print(person)

employee = Employee("Jindřich Sádlo", 28, "instalatér", 45000)
print(employee)


## Vícenásobná dědičnost

Python dovoluje vícenásobnou dědičnost dvojího druhu:

1. do šířky (potomek má více rodičů)
2. do hloubky (potomek může mít potomka)

Potomek podědí metody všech svých předků a může k nim přidat i nějaké vlastní.

In [ ]:
class BatteryDevice:
    def __init__(self, capacity):
        self.capacity = capacity
        self.status = 100
    
    def get_status():
        return self.status
    
class TouchscreenDevice:
    def __init__(self, size):
        self.size = size
        
    def show_mesage(self, message):
        print(f"message on screen: {message}")
        

class Smartphone(BatteryDevice, TouchscreenDevice):
    def __init__(self, brand, capacity, size):
        BatteryDevice.__init__(self, capacity)
        TouchscreenDevice.__init__(self, size)
        self.brand = brand
        
    def check_battery_status(self):
        if self.status < 20:
            self.show_mesage("Warning: battery low")
    
phone = Smartphone("Samsung", 6000, 6.5)
phone.check_battery_status()
phone.status = 19
phone.check_battery_status()

U více násobné dědičnosti je třeba dávat pozor na případy, kdy více rodičů implementuje tu samou metodu. Která varianta se pak zavolá?

In [ ]:
class A:
    def do_something(self):
        print("this is A")
        
class B:
    def do_something(self):
        print("this is B")

class C(A, B):
    pass

c = C()
c.do_something()

Python volá tu metodu, která je první nařadě podle seznamu, kterému říká MRO - Method Resolution Order, který získává pomocí tzv. C3 linearizace, což je algoritmus, který je zcela mimo rozsah tohoto předmětu (zjednodušeně: zleva doprava, zdola nahoru). Důležité je, že na pořádí se můžeme podívat, MRO se spočítá během definice třídy a najdeme ho pod atributem `__mro__`

In [ ]:
C.__mro__

Celá věc je poněkud matoucí. Následuje série podobných příkladů pouze s drobnými komentáři. Nejsnažší je si v konkrétních případech ověřit, že se třída chová, jak má, než se spoléhat na nějaký odhad.

Potřeba MRO je trochu patrnější, když přesuneme do konstruktoru. Následující příklad se chová stejně jako předchozí.

In [ ]:
class A:
    def __init__(self):
        print("this is A")
        
class B:
    def __init__(self):
        print("this is B")

class C(A, B):
    pass

c = C()

Ale to znamená, že rodič `B` není inicializovaný, což je nežádoucí. Můžeme zavolat konstruktory explicitně, ale to taky není žádoucí, neboť mimo to zhoršuje udržitelnost kódu (hard-coded názvy) a může narušit Liskov substitution principle (to L v SOLID, o tom jindy).

In [ ]:
class A:
    def __init__(self):
        print("this is A")
        
class B:
    def __init__(self):
        print("this is B")

class C(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
    
c = C()

Trochu lepší varianta je používat konstrukci `super().__init__()`. `super()` pracuje s MRO a zajišťuje, aby všechno bylo voláno jak má a aby nic nebylo voláno dvakrát, musíme ale `super()` volat ve všech zúčastněných třídách. (Všimněte si, že volání konstruktorů postupuje proti směru MRO - není moc těžké rozmyslet si proč).

In [ ]:
class A:
    def __init__(self):
        super().__init__()
        print("this is A")
        
class B:
    def __init__(self):
        super().__init__()
        print("this is B")

class C(A, B):
    def __init__(self):
        super().__init__()
    
c = C()

Prevence vícenásobného volání se projeví zejména do diamond dědičnosti (někdy diamond of death, příčina zhouby v mnoha inheritance based projektech). Přestože třídy `B` a `C` dědí z `A`, konstruktor `A` se správně zavolá jen jednou.

In [ ]:
class A:
    def __init__(self):
        print("this is A")
        
class B(A):
    def __init__(self):
        super().__init__()
        print("this is B")

class C(A):
    def __init__(self):
        super().__init__()
        print("this is C")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("this is D")
        
d = D()
D.__mro__

Vidíme, že při explicitním volání se konstruktor `A` zavolá dvakrát.

In [ ]:
class A:
    def __init__(self):
        print("this is A")
        
class B(A):
    def __init__(self):
        A.__init__(self)
        print("this is B")

class C(A):
    def __init__(self):
        A.__init__(self)
        print("this is C")

class D(B, C):
    def __init__(self):
        B.__init__(self)
        C.__init__(self)
        print("this is D")
        
d = D()
D.__mro__

Ohledně rozsáhlého využívání dědičnosti se vedou spory. Jsou tábory, které dědičnost zavrhují zcela, tábory které tvrdí, že bez dědičnosti nelze psát software a potom v podstatě všechny možné kombinace.

```{admonition} Osobní názor
:class: important
Osobně se příkláním k postoji, že trocha dědičnosti je dobrá, ale pokud začnete řešit problémy spojené s Method Resolution Order, zašli jste příliš daleko a Váš program je špatně navržený.
```

## Abstraktní třídy

Smysl abstraktní třídy je definovat jakýsi společný interface pro tématicky spřízněné třídy. K abstraktní třídě není možné stvořit instanci, neboť nic neimplementuje (nebo alespoň ne všechno).

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass
    
    @abstractmethod
    def perimeter(self):
        pass
    
    
class Rectangle(Shape):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def area(self):
        return self.a * self.b
    
    def perimeter(self):
        return 2 * (self.a + self.b)
    
    
# shape = Shape() # nelze
rect = Rectangle(2, 3)
print(rect.area())

Python dovoluje dědění z více abstraktních tříd najednou.

In [ ]:
class Building(ABC):
    @abstractmethod
    def can_i_get_in():
        pass
    
class Pentagon(Shape, Building):
    def __init__(self, a):
        self.a = a
        
    def area(self):
        return 4 # nechtelo se mi to hledat
    
    def perimeter(self):
        return 5 * self.a
    
    def can_i_get_in(self):
        return False
    
the_pentagon = Pentagon(3)
the_pentagon.can_i_get_in()

## Porovnávání typů v kontextu dědičnosti

Dosud jsme typy porovnávali pomocí zabudované funkce `type`, ale ta může být v kontextu potomků neodstatečná. Ukažme si dvě nové metody:

1. `isinstance(object, classinfo)` ověří, zda `object` je instancí třídy `classinfo` **nebo nějakého jejího potomka**
2. `issubclass(class, classinfo)` ověří, zda třída `class` je potomkem třídy `classinfo`

Následující příklad ilustruje, co to znamená oproti porovnávání `type`.

In [ ]:
class A:
    pass

class B(A):
    pass

a = A()
b = B()
isinstance(b, B), issubclass(B, A), isinstance(b, A), type(b) == B, type(b) == A

## Protocol (interface)

Některé jazyky (např. Java) zavádějí rozhraní (interface) - jakýsi kontrakt (sadu metod) pro třídy, které jej implementují. Zároveň interface nemusí tuto implementaci poskytovat.

```java
interface Drawable {
    void draw();
}

// Implement the interface in a class
class Circle implements Drawable {
    @Override
    public void draw() {
        System.out.println("Drawing a circle.");
    }
}
```

Jazyk C++ rozhraní nepodporuje a celou věc řeší pomocí abstraktních tříd.

```cpp
#include <iostream>

// Define the interface
class Drawable {
public:
    virtual void draw() = 0;
};

// Implement the interface in a class
class Circle : public Drawable {
public:
    void draw() override {
        std::cout << "Drawing a circle." << std::endl;
    }
};

```

V Pythonu můžeme použít abstraktní třídy podobně jako C++, ale obvyklejší je použít `Protocol`. Python je dynamicky typovaný, takže můžeme využívat *duck typing*.

> "If it looks like a duck and quacks like a duck, it's a duck"

V následujícím příkladu třída `Cabbage` není potomkem třídy `Drawable`, ale pouze implementuje její metody.

In [ ]:
from typing import Protocol

class Drawable(Protocol):
    def draw(self):
        ...
        
def draw_drawables(drawables: list[Drawable]):
    for d in drawables:
        d.draw()
        
class Cabbage:
    def draw(self):
        print(f"drawing {self.__class__.__name__}")
        
cabbage = Cabbage()

draw_drawables([cabbage])

Vzpomeňte si např. na context manager nebo iterátor. Context manager je cokoliv, co implementuje metody `__enter__` a `__exit__`, a iterátor cokoliv, co implementuje `__iter__` a `__next__`.

## Mixin

Mixin je příkladem, jak lze elegantně využít vícenásobnou dědičnost. Mixin je třída, která poskytuje metody pro použití v jiných třídách, ale není určena k samostatnému použití.

Výhody:
- jednoduché
- separuje zodpovědnost
- celkem flexibilní

Nevýhody:
- kolize názvů a komplikace spojené s vícenásobnou dědičností (lze vyřešit kompozicí)
- větší coupling mezi oddělenými třídami

In [ ]:
class LoggerMixin:
    def log(self, message):
        print(f"logging: {message}")
        
class Database:
    def connect(self):
        pass
    
    def close(self):
        pass
    
class LoggedDatabase(LoggerMixin, Database):
    def connect(self):
        self.log("connecting to database")
        Database.connect(self)
    
    def close(self):
        self.log("closing database")
        Database.close(self)
        
dlb = LoggedDatabase()
dlb.connect()
dlb.close()

### Composition vs. inheritance

Osobně se mi vícenásobná dědičnost příčí a snažím se jí vyhnout, jak to jen jde. Oblíbený způsob je nahrazovat dědičnost kompozicí. Tedy místo toho, aby třída podědila vlastnosti několika jiných tříd, udržuje si referenci na instance, které potřebuje. Například:

In [ ]:
class Logger:
    def __init__(self, log_strategies):
        self.log_strategies = log_strategies

    def log(self, message):
        for strategy in self.log_strategies:
            strategy.log(message)

class LogStrategy:
    def log(self, message):
        pass

class ConsoleLogStrategy(LogStrategy):
    def log(self, message):
        print(f"Console: {message}")

class FileLogStrategy(LogStrategy):
    def __init__(self, file_path):
        self.file_path = file_path

    def log(self, message):
        with open(self.file_path, "a") as log_file:
            log_file.write(f"File: {message}\n")

class DatabaseLogStrategy(LogStrategy):
    def log(self, message):
        # Code to connect and log message to a database
        pass

console_logger = Logger([ConsoleLogStrategy()])
console_and_file_logger = Logger([ConsoleLogStrategy(), FileLogStrategy("logfile.txt")])

console_logger.log("This is a console log message")
console_and_file_logger.log("This is a console and file log message")

Zvrácená dědičná implementace by mohl vypadat nějak takto:

In [ ]:
class Logger:
    pass

class FileLogger(Logger):
    pass

class ConsoleLogger(Logger):
    pass

class FileAndConsoleLogger(FileLogger, ConsoleLogger):
    pass